In [34]:
import logging
import progressbar
import os
from copy import deepcopy
import pandas as pd
from search_engine import Search

In [2]:
from sentence_transformers import SentenceTransformer, util

In [15]:
from dotenv import load_dotenv
from pathlib import Path

initial_wd = os.getcwd()
while os.path.split(os.getcwd())[-1] != 'scientific-knowledge-distiller':
    os.chdir(os.path.join(os.getcwd(), '..'))
root_path = os.getcwd()
os.chdir(initial_wd)

load_dotenv(dotenv_path=Path(os.path.join(root_path, '.env')))

True

In [37]:
query = 'out-of-distribution detection in deep neural networks'
limit = 1000
threshold_sim = 0.0

In [38]:
s = Search(query, limit=limit)
s.perform()

INFO:arxiv.arxiv:Requesting 100 results at offset 0
INFO:arxiv.arxiv:Requesting page of results
INFO:arxiv.arxiv:Got first page; 2071017 of inf results available
INFO:root:arXiv: 1
INFO:root:arXiv: 2
INFO:root:arXiv: 3
INFO:root:arXiv: 4
INFO:root:arXiv: 5
INFO:root:arXiv: 6
INFO:root:arXiv: 7
INFO:root:arXiv: 8
INFO:root:arXiv: 9
INFO:root:arXiv: 10
INFO:root:arXiv: 11
INFO:root:arXiv: 12
INFO:root:arXiv: 13
INFO:root:arXiv: 14
INFO:root:arXiv: 15
INFO:root:arXiv: 16
INFO:root:arXiv: 17
INFO:root:arXiv: 18
INFO:root:arXiv: 19
INFO:root:arXiv: 20
INFO:root:arXiv: 21
INFO:root:arXiv: 22
INFO:root:arXiv: 23
INFO:root:arXiv: 24
INFO:root:arXiv: 25
INFO:root:arXiv: 26
INFO:root:arXiv: 27
INFO:root:arXiv: 28
INFO:root:arXiv: 29
INFO:root:arXiv: 30
INFO:root:arXiv: 31
INFO:root:arXiv: 32
INFO:root:arXiv: 33
INFO:root:arXiv: 34
INFO:root:arXiv: 35
INFO:root:arXiv: 36
INFO:root:arXiv: 37
INFO:root:arXiv: 38
INFO:root:arXiv: 39
INFO:root:arXiv: 40
INFO:root:arXiv: 41
INFO:root:arXiv: 42
INFO:ro

In [39]:
results = list(s.results())

total found: 4504
starting deduplication...


In [40]:
final_results = []
model = SentenceTransformer('stsb-roberta-large')
embedding1 = model.encode(query, convert_to_tensor=True, normalize_embeddings=True)

for doc in progressbar.progressbar(results):
    if not doc.title:
        continue

    title = doc.title.lower()
    embedding2 = model.encode(title, convert_to_tensor=True, show_progress_bar=False, normalize_embeddings=True)
    cosine_scores = util.dot_score(embedding1, embedding2)
    sim_score = cosine_scores.item()
    #
    if sim_score > threshold_sim:
        final_results.append((deepcopy(doc), sim_score))
final_results = sorted(final_results, key=lambda x: -x[1])

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: stsb-roberta-large
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cpu


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

100% (4296 of 4296) |####################| Elapsed Time: 0:08:27 Time:  0:08:27


In [41]:
df = pd.DataFrame(final_results, columns=['Title', 'Sim'])

In [42]:
df

,Title,Sim
0,On Detection of Out of Distribution Inputs in ...,0.923167
1,Layer Adaptive Deep Neural Networks for Out-of...,0.906879
2,Out-of-Distribution Detection Using Neural Ren...,0.899319
3,Layer Adaptive Deep Neural Networks for Out-of...,0.897021
4,Out-of-Distribution Example Detection in Deep ...,0.896056
...,...,...
4254,Toward Explainable Fashion Recommendation,0.058825
4255,Thermal-based early breast cancer detection us...,0.048570
4256,Combating Word-level Adversarial Text with Rob...,0.048486
4257,Are they Our Brothers? Analysis and Detection ...,0.029908
